## load 参数

In [1]:
import numpy as np
import cv2
import scipy.io as scio

Params=scio.loadmat('workspace.mat')
K = np.array(Params['K'])
K = K.T
distortion=np.array(Params['distortion'])

## sift特征匹配

In [2]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

imgpath = 'images/'
MIN_MATCH_COUNT = 10

def match_pts(imgname1, imgname2):
    img1 = cv2.imread(imgpath+imgname1, 0)          # queryImage
    img2 = cv2.imread(imgpath+imgname2, 0)        # trainImage

    # Initiate SIFT detector
    sift = cv2.xfeatures2d.SIFT_create()

    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(img1,None)
    kp2, des2 = sift.detectAndCompute(img2,None)

    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks = 50)

    flann = cv2.FlannBasedMatcher(index_params, search_params)

    matches = flann.knnMatch(des1,des2,k=2)

    # store all the good matches as per Lowe's ratio test.
    good = []
    pts1 = []
    pts2 = []
    
    # ratio test as per Lowe's paper
    for i,(m,n) in enumerate(matches):
        if m.distance < 0.8*n.distance:
            good.append(m)
            pts2.append(kp2[m.trainIdx].pt)
            pts1.append(kp1[m.queryIdx].pt)

    pts1 = np.int32(pts1)
    pts2 = np.int32(pts2)
    # return the np.array type
    return pts1, pts2

# estimate估计参数

In [15]:
from numpy import *
from scipy import *
from scipy.linalg import *
from pylab import *
import cv2
import sys

def find_fundamental(pts1, pts2):
    F, mask = cv2.findFundamentalMat(pts1,pts2,cv2.FM_LMEDS)
    return F
    
def find_essential(F, K1, K2):
    return K2.T @ F @ K1

def skew(a):
    #Skew matrix A such that cross(a,v) = Av for any v
    return array([[0,-a[2],a[1]],[a[2],0,-a[0]],[-a[1],a[0],0]])

def anti_skew(A):
    return array([A[2,1], A[2,0], A[1,0]])

def triangulate(P1, P2, pts1, pts2):
    n1 = pts1.shape[0]
    n2 = pts2.shape[0]
    
    p1 = vstack([ pts1.T, ones(n1) ]).T
    p2 = vstack([ pts2.T, ones(n2) ]).T
    
    P = zeros((n1, 4))
    for i in range(n1):
        cross1 = skew(p1[i])
        cross2 = skew(p2[i])
        Q = vstack([cross1 @ P1, cross2 @ P2])
        U,S,VT = svd(Q)
        z = VT[-1, :]
        P[i, :] = z/z[3]
    
    return P[:, 0:3]

def guess_rotation(E):
    # make sure E is rank 2
    U,S,V = svd(E)
    if det(dot(U,V)) < 0:
        V = -V
    E = dot(U,dot(diag([1,1,0]),V))    

    W = array([[0,-1,0],[1,0,0],[0,0,1]])
        
    R21 = dot(U,dot(W,V))
    R22 = dot(U,dot(W.T,V))
    
    T21 = anti_skew(E @ R21.T) 
    T22 = anti_skew(E @ R22.T)
    
    return R21, T21, R22, T22

def test_rotation(P1, P2, pts1, pts2):
    #temp = cv2.triangulatePoints(P1, P2, transpose(pts1), transpose(pts2))
    temp = triangulate(P1, P2, pts1, pts2)
    for v in temp:
        if v[-1] < 0:
            return False
    return True

def errnum_rotation(P1, P2, pts1, pts2):
    #temp = cv2.triangulatePoints(P1, P2, transpose(pts1), transpose(pts2))
    temp = triangulate(P1, P2, pts1, pts2)
    num = 0
    for v in temp:
        if v[-1] < 0:
            num += 1
    return num
    

def find_rotation(E, K1, K2, pts1, pts2):
    R1 = eye(3)
    T1 = array([0, 0, 0])
    P1 = K1 @ vstack([R1.T, T1]).T
    
    R21, T21, R22, T22 = guess_rotation(E)
    
    P21 = K2 @ vstack([R21.T, T21]).T
    P22 = K2 @ vstack([R22.T, T22]).T

    #key1 = test_rotation(P1, P21, pts1, pts2)
    #key2 = test_rotation(P1, P22, pts1, pts2)
    
    
    errn1 = errnum_rotation(P1, P21, pts1, pts2)
    errn2 = errnum_rotation(P1, P22, pts1, pts2)
    
    R2 = R21
    T2 = T21

    if T21[1] < 0:
        R2 = R21
        T2 = T21
    elif T22[1] < 0:
        R2 = R22
        T2 = T22
    else:
        print('Can not find the right R matrix.')
        sys.exit()

    return R1, T1, R2, T2
    
def find_pose(R1, T1, R2, T2, K1, K2):
    #return K @ vstack([R.T, T]).T
    P1 = K1 @ vstack([R1.T, T1]).T
    P2 = K2 @ vstack([R2.T, T2]).T
    
    return P1, P2


## 运行

In [16]:
pts1, pts2 = match_pts('1_.jpg', '2_.jpg')
print(pts1.shape)
F = find_fundamental(pts1, pts2)
E = find_essential(F, K, K)
R1, T1, R2, T2 = find_rotation(E, K, K, pts1, pts2)

print(R1, T1, R2, T2, sep='\n')
K1 = K
K2 = K
P1, P2 = find_pose(R1, T1, R2, T2, K1, K2)
points = triangulate(P1, P2, pts1, pts2)

(1340, 2)
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[0 0 0]
[[ 9.33062652e-01 -3.86475291e-03 -3.59693134e-01]
 [-3.86090148e-03 -9.99992281e-01  7.29121780e-04]
 [-3.59693175e-01  7.08423451e-04 -9.33070371e-01]]
[ 0.9751416  -0.00491469 -0.22152812]


In [18]:
print(K, F, E, sep='\n')

[[966.23930753   0.         583.67813597]
 [  0.         964.93489257 723.88506865]
 [  0.           0.           1.        ]]
[[ 1.75403911e-07  1.56012202e-05 -1.11554963e-02]
 [-9.83158864e-06  2.27223274e-07 -5.96975343e-02]
 [ 7.55013245e-03  5.70905777e-02  1.00000000e+00]]
[[  0.16376032  14.54592283   0.2322583 ]
 [ -9.16656054   0.21156744 -62.98278073]
 [  0.51749039  64.03419144  -0.37492419]]


In [ ]:
print(K.T)

## MVS多图

In [5]:
def sparse(imgname1, imgname2, K1, K2):
    pts1, pts2 = match_pts(imgname1, imgname2)
    F = find_fundamental(pts1, pts2)
    E = find_essential(F, K1, K2)
    
    R1, T1, R2, T2 = find_rotation(E, K1, K2, pts1, pts2)
    
    P1, P2 = find_pose(R1, T1, R2, T2, K1, K2)
    
    X = triangulate(P1, P2, pts1, pts2)
    R = R2
    T = T2
    
    return X, R, T

## 多图融合

In [6]:
def multi_sparse(imgs, K):
    n = len(imgs)
    xn = n-1
    Xs = []
    Rs = []
    Ts = []
    for i in range(xn):
        Xtemp, Rtemp, Ttemp = sparse(imgs[i], imgs[i+1], K, K)
        print(i, Xtemp[10:20,])
        print(Rtemp, Ttemp)
        
        Xs.append(Xtemp)
        Rs.append(Rtemp)
        Ts.append(Ttemp)
    
    X = Xs[0]

    for i in range(1, xn):
        newX = Xs[i]
        nn = newX.shape[0]
        newX = newX.T
        for j in range(1, i + 1):
            newX = vstack([Rs[i-j].T, Ts[i-j]]).T @ vstack([newX, ones(nn)])
        newX = newX.T
        X = vstack([X, newX])
        
    return X
    
imgs = ['1_.jpg', '2_.jpg', '3_.jpg', '4_.jpg', '5_.jpg', '6_.jpg']
X = multi_sparse(imgs, K)
print(X.shape)

0 [[-0.35661128 -0.43659305  0.93034978]
 [-0.34301304 -0.34980693  0.92813531]
 [-0.33386419  0.39211506  0.92167546]
 [-0.34851695 -0.29648506  0.90495577]
 [-0.34077716 -0.31871689  0.93038593]
 [-0.33400331  0.40028925  0.92735175]
 [-0.33617337  0.16014765  0.93541626]
 [-0.32150419  0.25339911  0.90837155]
 [-0.32938769  0.13424337  0.93105858]
 [-0.34082633 -0.38378825  0.95494034]]
[[ 9.33062652e-01 -3.86475291e-03 -3.59693134e-01]
 [-3.86090148e-03 -9.99992281e-01  7.29121780e-04]
 [-3.59693175e-01  7.08423451e-04 -9.33070371e-01]] [ 0.9751416  -0.00491469 -0.22152812]
1 [[-0.10613065 -0.37279484  0.86469135]
 [-0.23911042  0.39173244  1.0905539 ]
 [-0.23924466  0.3911083   1.09145536]
 [-0.17524269  0.33962564  1.14717233]
 [ 0.0930517   0.15100196 -0.02098849]
 [ 0.08578946  0.07943422  0.8008088 ]
 [ 0.17179039 -0.50545147  2.01148774]
 [-0.015212    0.13206703  0.63097948]
 [ 0.08417817  0.08947772  0.77294967]
 [-0.01458069  0.1346579   0.63227172]]
[[-0.92137975 -0.37791

In [8]:
print(X[1500:1510,], X[1800:1810,], X[3020:3030, ], sep='\n*****')

[[ 1.01980765  0.07912635 -2.14297328]
 [ 2.1722339   1.93826082 -6.3475416 ]
 [ 2.15068546  1.82189243 -6.21985706]
 [ 1.05055265  1.7206494  -1.88545818]
 [ 1.26570233  0.51957055 -3.11421544]
 [ 1.83031187 -0.05767351 -1.93718502]
 [ 1.24811885  0.34212518 -2.94831496]
 [ 1.24089823  0.3381152  -2.93857584]
 [ 1.16119873  0.27030212 -2.68930951]
 [ 0.83280439  3.7294199  -2.24390806]]
*****[[ 1.10714585 -0.99473991 -0.02965354]
 [ 1.93984145 -3.2908222   0.39697951]
 [ 1.39921916 -1.79613091 -0.65142684]
 [ 1.40193802 -1.92511988 -0.29313513]
 [ 1.31545424 -1.45893168  0.02996553]
 [ 1.30839852 -1.39793984  0.06592661]
 [ 1.23420784 -1.17298257 -0.56731267]
 [ 1.35425861 -1.58297569  0.00566968]
 [ 1.40946688 -2.60373297  0.19482329]
 [ 1.30784672 -1.37556511  0.0591723 ]]
*****[[  6.04001027 -11.77197505  20.43885036]
 [  5.46168611  -7.56176639   4.2175497 ]
 [ -1.86066101  -0.37582184  -1.96686652]
 [  5.51270844  -8.05963271   6.23281052]
 [  5.28813097  -6.80926177   1.51617209

In [7]:
print(F)

[[ 1.75403911e-07  1.56012202e-05 -1.11554963e-02]
 [-9.83158864e-06  2.27223274e-07 -5.96975343e-02]
 [ 7.55013245e-03  5.70905777e-02  1.00000000e+00]]


In [10]:
Farr = array(F)
print(Farr)
print(trace(Farr.T @ Farr))

[[ 1.75403911e-07  1.56012202e-05 -1.11554963e-02]
 [-9.83158864e-06  2.27223274e-07 -5.96975343e-02]
 [ 7.55013245e-03  5.70905777e-02  1.00000000e+00]]
1.0070045796023517


In [9]:
Earr = array(E)
print(Earr)
print(trace(Earr.T @ Earr))

[[  0.16376032  14.54592283   0.2322583 ]
 [ -9.16656054   0.21156744 -62.98278073]
 [  0.51749039  64.03419144  -0.37492419]]
8363.351932203246


In [11]:
X, R, T = sparse('2_.jpg', '3_.jpg', K, K)

In [12]:
print(X.shape)

(143, 3)


In [13]:
print(X)

[[-2.42151573e-01  5.95389512e-02  6.39988701e-01]
 [ 5.67823001e-02  3.35428729e-01  4.40884117e-02]
 [-1.77519823e-01 -4.72423955e-02  4.43758403e-01]
 [-4.27837967e-01 -6.31052315e-01  2.06986784e+00]
 [-1.05360140e-01  8.29458823e-02  6.72495040e-01]
 [-7.85942680e+00 -1.89674734e+01  4.16278365e+01]
 [ 4.93485979e-02  7.87541169e-02  1.66211465e-01]
 [ 4.53646277e-02  9.18034569e-02  1.66940062e-01]
 [ 4.53646277e-02  9.18034569e-02  1.66940062e-01]
 [ 4.76910202e-02 -2.04314454e-01  1.47332480e+00]
 [-1.06130648e-01 -3.72794845e-01  8.64691352e-01]
 [-2.39110417e-01  3.91732440e-01  1.09055390e+00]
 [-2.39244659e-01  3.91108302e-01  1.09145536e+00]
 [-1.75242693e-01  3.39625644e-01  1.14717233e+00]
 [ 9.30516980e-02  1.51001964e-01 -2.09884890e-02]
 [ 8.57894616e-02  7.94342187e-02  8.00808799e-01]
 [ 1.71790388e-01 -5.05451467e-01  2.01148774e+00]
 [-1.52120018e-02  1.32067033e-01  6.30979482e-01]
 [ 8.41781732e-02  8.94777204e-02  7.72949668e-01]
 [-1.45806863e-02  1.34657898e-

In [14]:
print(T)

[0.03687106 0.54088425 0.84028849]
